# RDP over Reverse SSH Tunnel
Detects svchost hosting RDP termsvcs communicating with the loopback address and on TCP port 3389

## Rule Content
```
- title: RDP over Reverse SSH Tunnel
  id: 5f699bc5-5446-4a4a-a0b7-5ef2885a3eb4
  status: experimental
  description: Detects svchost hosting RDP termsvcs communicating with the loopback
    address and on TCP port 3389
  references:
  - https://twitter.com/SBousseaden/status/1096148422984384514
  author: Samir Bousseaden
  date: 2019/02/16
  tags:
  - attack.defense_evasion
  - attack.command_and_control
  - attack.t1076
  - car.2013-07-002
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 3
      Image: '*\svchost.exe'
      Initiated: 'true'
      SourcePort: 3389
      DestinationIp:
      - 127.*
      - ::1
    condition: selection
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"3" AND process_path.keyword:*\\svchost.exe AND network_initiated:"true" AND src_port:"3389" AND dst_ip_addr.keyword:(127.* OR \:\:1))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()